Hi Everyone

I’m Fateme, an environmental specialist passionate about air quality modeling.  
While working on atmospheric dispersion modeling with AERMET, I faced challenges processing raw meteorological data to meet input requirements.  
To solve this, I developed the scripts below to clean, organize, and prepare the data — filling gaps, aligning timestamps, and converting intervals to hourly records.  
I’m sharing this in case it helps others working on similar projects...

**Importing Libraries**

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime, timedelta
import glob
import os
import re

**Filling Blank Cells**

In [ ]:
# Path to the Excel file
file_path = '/content/drive/MyDrive/Your_File_Name.xlsx' #Update Your Path Here

# Load the Excel file
df = pd.read_excel(file_path)

# Define the column names (make sure they match exactly)
temperature_col = 'Temperture' #Update Your Parametr Name Here
wind_dir_col = 'Wind Direction' #Update Your Parametr Name Here

# Function to fill missing values
def fill_missing_values(df):
    # Fill temperature blanks with 0
    if temperature_col in df.columns:
        df[temperature_col].fillna(0, inplace=True)

    # Fill wind direction blanks with the most frequent value
    if wind_dir_col in df.columns:
        most_frequent_wind_dir = df[wind_dir_col].mode()[0]
        df[wind_dir_col].fillna(most_frequent_wind_dir, inplace=True)

    # Fill other columns' blanks with their mean
    for col in df.columns:
        if col not in [temperature_col, wind_dir_col]:
            if df[col].dtype in [np.float64, np.int64]:
                df[col].fillna(df[col].mean(), inplace=True)

    return df

# Fill missing values
filled_df = fill_missing_values(df)

# Save the cleaned file back to Google Drive
output_path = '/content/drive/My Drive/Your_File_Name.xlsx' #Update Your Path Here
filled_df.to_excel(output_path, index=False)

print(f"Cleaned file saved to: {output_path}")

<ipython-input-14-4ed08e3fb428>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[temperature_col].fillna(0, inplace=True)
<ipython-input-14-4ed08e3fb428>:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

Cleaned file saved to: /content/drive/My Drive/AERMET_Data/Sarcheshme/1396_1402_1/1402_Fill.xlsx


**Merging Excel Files from 1396_1402**

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Path to the folder containing the Excel files (inside Google Drive)
folder_path = '/content/drive/MyDrive/' #Update Your Path Here

# List all Excel files in the folder
files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# Function to extract year from the filename (assuming the filename is the year)
def extract_year_from_filename(filename):
    match = re.search(r'14\d{2}|13\d{2}', filename)
    return int(match.group()) if match else 0

# Sort files by year
files = sorted(files, key=lambda x: extract_year_from_filename(os.path.basename(x)))

# List to store dataframes
dfs = []

# Read and append each file to the list
for i, file in enumerate(files):
    df = pd.read_excel(file)
    dfs.append(df)
    print(f"File {os.path.basename(file)} added")

# Merge all dataframes, keeping headers only for the first file
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged file back to Google Drive
output_path = os.path.join(folder_path, 'Your_File_Name.xlsx')
merged_df.to_excel(output_path, index=False)

print(f"All files have been merged in chronological order and saved to:\n{output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File 1396_Fill.xlsx added
File 1397_Fill.xlsx added
File 1398_Fill.xlsx added
File 1399_Fill.xlsx added
File 1400_Fill.xlsx added
File 1401_Fill.xlsx added
File 1402_Fill.xlsx added
All files have been merged in chronological order and saved to:
/content/drive/My Drive/AERMET_Data/Sarcheshme/1396_1402_1/Fill/1396_1402_merged.xlsx


**Converting Date Column to Seprate Year/Month/Day/Hour/Minute Columns**

In [ ]:

# Path to the merged file
folder_path = '/content/drive/MyDrive/' #Update Your Path Here
input_file = os.path.join(folder_path, 'Your_File_Name.xlsx')

# Load the merged file
df = pd.read_excel(input_file)

# Check the first few rows to verify the date column name
print(df.head())

# Split the 'تاریخ' column into separate date and time components
df['Year'] = df['تاریخ'].str.split(' ').str[1].str.split('/').str[0]
df['Month'] = df['تاریخ'].str.split(' ').str[1].str.split('/').str[1]
df['Day'] = df['تاریخ'].str.split(' ').str[1].str.split('/').str[2]
df['Hour'] = df['تاریخ'].str.split(' ').str[0].str.split(':').str[0]
df['Minute'] = df['تاریخ'].str.split(' ').str[0].str.split(':').str[1]

# Save the updated DataFrame
output_file = os.path.join(folder_path, 'Your_File_Name.xlsx')
df.to_excel(output_file, index=False)

print(f"Date column split into Year, Month, Day, Hour, and Minute! Saved to:\n{output_file}")


   رطوبت هوا (2 متر) ( % )   سرعت باد (10 متر) ( m/s )   \
0                      87.1                         9.7   
1                      76.6                         9.5   
2                      76.1                         9.2   
3                      78.6                         7.7   
4                      83.3                         7.3   

  دمای هوا (2 متر) ( °C )   جهت باد (10 متر) ( Deg )   باران ( mm )   \
0                      5.7                        237            0.0   
1                      5.9                        236            0.0   
2                        6                        230            0.0   
3                      6.2                        231            0.0   
4                      6.9                        213            0.0   

   فشار اتمسفر ( hPa )                 تاریخ  
0                 744.0  10:50:00 1396/01/01  
1                 744.0  11:00:00 1396/01/01  
2                 744.0  11:10:00 1396/01/01  
3                 743.9 

**Converting Data from Every 10Min to Every 1Hour Intervals**

In [ ]:
# Path to the split date file
folder_path = '/content/drive/MyDrive/' #Update Your Path Here
input_file = os.path.join(folder_path, 'Your_File_Name.xlsx')

# Load the split date file
df = pd.read_excel(input_file)

# Check the first few rows to verify the data
print(df.head())

# Filter rows where 'Minute' is 0
hourly_df = df[df['Minute'] == 0]

# Drop the 'Minute' column since it's no longer needed
hourly_df = hourly_df.drop(columns=['Minute'])

# Save the updated DataFrame
output_file = os.path.join(folder_path, 'Your_File_Name.xlsx')
hourly_df.to_excel(output_file, index=False)

print(f"Data filtered to keep only hourly records! Saved to:\n{output_file}")


   Year  Month  Day  Hour  Minute  رطوبت هوا (2 متر) ( % )   \
0  1396      1    1    10      50                      87.1   
1  1396      1    1    11       0                      76.6   
2  1396      1    1    11      10                      76.1   
3  1396      1    1    11      20                      78.6   
4  1396      1    1    11      30                      83.3   

   سرعت باد (10 متر) ( m/s )   دمای هوا (2 متر) ( °C )   \
0                         9.7                       5.7   
1                         9.5                       5.9   
2                         9.2                       6.0   
3                         7.7                       6.2   
4                         7.3                       6.9   

   جهت باد (10 متر) ( Deg )   باران ( mm )   فشار اتمسفر ( hPa )   
0                        237            0.0                 744.0  
1                        236            0.0                 744.0  
2                        230            0.0                 74

**Filling missed Hours**

In [ ]:
# Function to fill missing values in the DataFrame
def fill_missing_values(df):
    # Fill missing temperature with 0
    df['Temperture'].fillna(0, inplace=True) #Update Your Parameter Name Here

    # Fill missing wind direction with the most frequent value
    if df['Wind Direction'].notna().any():   #Update Your Parameter Name Here
        most_common_wind_dir = df['Wind Direction'].mode()[0]
        df['Wind Direction'].fillna(most_common_wind_dir, inplace=True)

    # Fill other missing values with the column mean
    for col in df.columns:
        if col not in ['Year', 'Month', 'Day', 'Hour', 'Temperture', 'Wind Direction']:
            df[col].fillna(df[col].mean(), inplace=True)

    return df

# Function to generate complete Shamsi date range
def generate_complete_dates(years):
    date_rows = []
    for year in years:
        for month in range(1, 13):
            days_in_month = 31 if month <= 6 else 30
            for day in range(1, days_in_month + 1):
                for hour in range(0, 24):
                    date_rows.append([year, month, day, hour])

    return pd.DataFrame(date_rows, columns=['Year', 'Month', 'Day', 'Hour'])

# Load the data file from Google Drive
file_path = '/content/drive/My Drive/Your_File_Name.xlsx' #Update Your Path Here
df = pd.read_excel(file_path)

# Get unique years from the data
unique_years = df['Year'].unique()

# Create the complete date range
complete_dates_df = generate_complete_dates(unique_years)

# Merge the complete date range with the existing data
merged_df = pd.merge(complete_dates_df, df, on=['Year', 'Month', 'Day', 'Hour'], how='left')

# Fill missing values
filled_df = fill_missing_values(merged_df)

# Save the final DataFrame to Google Drive
output_path = '/content/drive/My Drive/Your_File_Name.xlsx'  #Update Your Path Here
filled_df.to_excel(output_path, index=False)

print("Missing dates added, gaps filled, and final dataset saved to Google Drive!")


<ipython-input-23-5bda9ac78407>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['دمای هوا (2 متر) ( °C ) '].fillna(0, inplace=True)
<ipython-input-23-5bda9ac78407>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

Missing dates added, gaps filled, and final dataset saved to Google Drive!
